# DOSAGE Dataset – Filtering Demo

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load CSVs (replace with correct file paths if needed)
d_dose = pd.read_csv('d_dose.csv')
s_dose = pd.read_csv('s_dose.csv')
r_dose = pd.read_csv('r_dose.csv')

Filter Function: d_dose.csv (Disease-Specific Dosage)

In [3]:
def filter_d_dose(generic=None, disease=None, age=None, route=None, age_unit='y'):
    df = d_dose.copy()

    if generic:
        df = df[df['generic'].str.lower() == generic.lower()]
    if disease:
        df = df[df['disease'].str.lower() == disease.lower()]
    if route:
        df = df[df['route'].str.upper() == route.upper()]

    if age is not None:
        age_y = age if age_unit == 'y' else age / 12 if age_unit == 'm' else age / 365
        age_m = age if age_unit == 'm' else age * 12 if age_unit == 'y' else age / 30.44
        age_d = age if age_unit == 'd' else age * 365 if age_unit == 'y' else age * 30.44

        def age_match(row):
            for min_col, max_col, val in [
                ('min_age_y', 'max_age_y', age_y),
                ('min_age_m', 'max_age_m', age_m),
                ('min_age_d', 'max_age_d', age_d)
            ]:
                if pd.notna(row[min_col]) and pd.notna(row[max_col]):
                    if float(row[min_col]) <= val <= float(row[max_col]):
                        return True
            return False

        df = df[df.apply(age_match, axis=1)]

    return df.reset_index(drop=True)


In [4]:
# ▶ Example usage:
filter_d_dose(generic='Ampicillin', disease='Gastroenteritis', age=12, route='PO')

,generic,disease,min_age_d,max_age_d,min_age_m,max_age_m,min_age_y,max_age_y,min_weight,max_weight,...,min_dose_dw_iu,max_dose_dw_iu,limit_mg,limit_iu,min_dose_dd_mg,max_dose_dd_mg,min_dose_dd_UNIT,max_dose_dd_iu,route,Unnamed: 21
0,Ampicillin,Gastroenteritis,NaN,NaN,NaN,NaN,12.0,120.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2000.0,2500.0,NaN,NaN,PO,NaN


Filter Function: s_dose.csv (Standard Dosage)

In [5]:
def filter_s_dose(generic=None, age=None, route=None, age_unit='y'):
    df = s_dose.copy()

    if generic:
        df = df[df['generic'].str.lower() == generic.lower()]
    if route:
        df = df[df['route'].str.upper() == route.upper()]

    if age is not None:
        age_y = age if age_unit == 'y' else age / 12 if age_unit == 'm' else age / 365
        age_m = age if age_unit == 'm' else age * 12 if age_unit == 'y' else age / 30.44
        age_d = age if age_unit == 'd' else age * 365 if age_unit == 'y' else age * 30.44

        def age_match(row):
            for min_col, max_col, val in [
                ('min_age_y', 'max_age_y', age_y),
                ('min_age_m', 'max_age_m', age_m),
                ('min_age_d', 'max_age_d', age_d)
            ]:
                if pd.notna(row[min_col]) and pd.notna(row[max_col]):
                    if float(row[min_col]) <= val <= float(row[max_col]):
                        return True
            return False

        df = df[df.apply(age_match, axis=1)]

    return df.reset_index(drop=True)


In [7]:
# ▶ Example usage:
filter_s_dose(generic='Cipro', age=25, route='PO')

""


 Filter Function: r_dose.csv (Renal Adjustment Dosage)

In [8]:
def filter_r_dose(generic=None, disease=None, crcl=None, route=None, weight=None):
    df = r_dose.copy()

    if generic:
        df = df[df['generic'].str.lower() == generic.lower()]
    if disease:
        df = df[df['disease'].str.lower() == disease.lower()]
    if route:
        df = df[df['route'].str.upper() == route.upper()]
    if crcl is not None:
        df = df[
            (df['min_crcl'].astype(float) <= crcl) &
            (df['max_crcl'].astype(float) >= crcl)
        ]
    if weight is not None:
        df = df[
            ((df['min_weight'].isna()) | (df['min_weight'].astype(float) <= weight)) &
            ((df['max_weight'].isna()) | (df['max_weight'].astype(float) >= weight))
        ]

    return df.reset_index(drop=True)

In [11]:
# ▶ Example usage:
filter_r_dose(generic='Ciprofloxacin', crcl=25, route='IV')

,generic,disease,min_weight,max_weight,min_crcl,max_crcl,max_dose_dd_mg,max_dose_dw_mg,max_dose_dd_iu,max_dose_dw_iu,route,flag
0,Ciprofloxacin,NaN,NaN,NaN,5.0,29.0,400.0,NaN,NaN,NaN,IV,NaN
